In [1]:
import yaml 
import os

with open(os.path.join("..", "keys.yaml"), "r") as file:
    config = yaml.safe_load(file)
    openai_api_key = config["openai_api_key"]

In [2]:
from explingo import Narrator, Grader 

example_narratives = [
    ("(Above ground living area square feet, 1256.00, -12527.46), (Overall material and finish of the house, 5.00, -10743.76), (Second floor square feet, 0.00, -10142.29)", 
     "The house's living area size of around 1,200 sq. ft., lower quality materials (5/10), and lack of a second floor all reduce the house's value."),
    ("(Second floor square feet, 854.00, 12757.84), (Original construction date, 2003.00, 9115.72)",
     "The house's large second floor of around 850 sq. ft and recent construction date of 2003 increase its value."),
    ("(Overall material and finish of the house, 8.00, 10743.76), (Above ground living area square feet, 2000.00, 12527.46), (Second floor square feet, 1000.00, 10142.29)",
        "The house's high quality materials (8/10), large living area size of around 2,000 sq. ft., and a second floor of around 1,000 sq. ft. all increase the house's value."),
]

explanation_format = "(feature name, feature value, SHAP feature contribution)"
context = "The model predicts house prices"

narrator = Narrator(openai_api_key=openai_api_key,
                    explanation_format=explanation_format,
                    context=context,
                    sample_narratives=example_narratives)


In [3]:
explanation = "(number of rooms, 11, 7020), (fireplace, yes, 12903)"

narrative = narrator.narrate(explanation)
narrative

"The house's large number of rooms (11) and the presence of a fireplace both increase its value."

In [4]:
grader = Grader(openai_api_key=openai_api_key, 
                metrics="all", 
                sample_narratives=example_narratives)

grader(explanation=explanation, explanation_format=explanation_format, narrative=narrative)

accuracy        4.0
completeness    4.0
fluency         4.0
conciseness     4.0
dtype: float64

In [5]:
narrative_with_bootstrap = narrator.narrate(explanation, n_bootstrapped=1, grader=grader)
narrative_with_bootstrap

  0%|          | 0/3 [00:00<?, ?it/s]

TRACE [(Predict(NarratorSig(context, explanation, explanation_format -> narrative
    instructions="You are helping users understand an ML model's prediction. Given an explanation and information about the model,\nconvert the explanation into a human-readable narrative."
    context = Field(annotation=str required=True json_schema_extra={'desc': 'what the ML model predicts', '__dspy_field_type': 'input', 'prefix': 'Context:'})
    explanation = Field(annotation=str required=True json_schema_extra={'desc': "explanation of an ML model's prediction", '__dspy_field_type': 'input', 'prefix': 'Explanation:'})
    explanation_format = Field(annotation=str required=True json_schema_extra={'desc': 'format the explanation is given in', '__dspy_field_type': 'input', 'prefix': 'Explanation Format:'})
    narrative = Field(annotation=str required=True json_schema_extra={'desc': 'human-readable narrative version of the explanation', '__dspy_field_type': 'output', 'prefix': 'Narrative:'})
)), {'expla

  0%|          | 0/3 [00:00<?, ?it/s]


Bootstrapped 1 full traces after 1 examples in round 2.


"The house's large number of rooms, totaling 11, and the presence of a fireplace both contribute to an increase in its value."

In [6]:
grader(explanation=explanation, explanation_format=explanation_format, narrative=narrative_with_bootstrap)

accuracy        4.0
completeness    4.0
fluency         4.0
conciseness     4.0
dtype: float64

In [7]:
narrator.llm.inspect_history()




You are helping users understand an ML model's prediction. Given an explanation and information about the model,
convert the explanation into a human-readable narrative.

---

Follow the following format.

Context: what the ML model predicts

Explanation: explanation of an ML model's prediction

Explanation Format: format the explanation is given in

Narrative: human-readable narrative version of the explanation

---

Context: The model predicts house prices

Explanation: (Above ground living area square feet, 1256.00, -12527.46), (Overall material and finish of the house, 5.00, -10743.76), (Second floor square feet, 0.00, -10142.29)

Explanation Format: (feature name, feature value, SHAP feature contribution)

Narrative: Narrative: The house's relatively small above ground living area of around 1,256 sq. ft., average quality materials (5/10), and lack of a second floor all decrease the house's value.

---

Context: The model predicts house prices
Explanation: (Overall material and 

"\n\n\nYou are helping users understand an ML model's prediction. Given an explanation and information about the model,\nconvert the explanation into a human-readable narrative.\n\n---\n\nFollow the following format.\n\nContext: what the ML model predicts\n\nExplanation: explanation of an ML model's prediction\n\nExplanation Format: format the explanation is given in\n\nNarrative: human-readable narrative version of the explanation\n\n---\n\nContext: The model predicts house prices\n\nExplanation: (Above ground living area square feet, 1256.00, -12527.46), (Overall material and finish of the house, 5.00, -10743.76), (Second floor square feet, 0.00, -10142.29)\n\nExplanation Format: (feature name, feature value, SHAP feature contribution)\n\nNarrative: Narrative: The house's relatively small above ground living area of around 1,256 sq. ft., average quality materials (5/10), and lack of a second floor all decrease the house's value.\n\n---\n\nContext: The model predicts house prices\nExp